In [1]:
!kaggle datasets download -d vivek468/beginner-chest-xray-image-classification

Dataset URL: https://www.kaggle.com/datasets/vivek468/beginner-chest-xray-image-classification
License(s): other
100% 2.08G/2.08G [01:50<00:00, 20.8MB/s]
100% 2.08G/2.08G [01:50<00:00, 20.1MB/s]


In [2]:
import zipfile

# ZIP dosyasının yolu
zip_file_path = '/content/beginner-chest-xray-image-classification.zip'

# Çıkartılacak hedef klasör
extracted_folder_path = '/content/'

# ZIP dosyasını aç
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [14]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Veri setinin dizini
data_dir = "/content/chest_xray"

# Veri ön işleme ve artırma
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet mean ve std
])

# Veri yükleyicileri oluşturma
train_dataset = datasets.ImageFolder(root=data_dir + '/train', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = datasets.ImageFolder(root=data_dir + '/test', transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [16]:
import torch.nn as nn
import torchvision.models as models

# VGG19 modelini yükleme
resnet50 = models.resnet50(pretrained=True)

# Son fully connected layer'ı değiştirme
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Eğitim için CUDA kullanılabilirse GPU'ya taşıma ve DataParallel kullanarak sarmalama
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = nn.DataParallel(resnet50)  # DataParallel ile modeli sarmalama
resnet50 = resnet50.to(device)

In [19]:
import torch.optim as optim
import torch.nn.functional as F

# Optimizasyon algoritması ve kayıp fonksiyonu
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Eğitim döngüsü
from tqdm import tqdm

epochs = 1
for epoch in range(epochs):
    running_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch", colour='green', leave=True)
    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Gradientleri sıfırlama
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        # Backward pass ve optimizasyon
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        progress_bar.set_postfix({'Loss': loss.item()})

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/2: 100%|██████████| 229/229 [03:30<00:00,  1.09batch/s, Loss=0.0125]


Epoch 1/2, Loss: 0.0550


Epoch 2/2: 100%|██████████| 229/229 [03:31<00:00,  1.08batch/s, Loss=0.0185]

Epoch 2/2, Loss: 0.0202


In [20]:
correct = 0
total = 0
TP = 0
FP = 0
FN = 0
TN = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet50(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Kesinlik ve Duyarlılık için TP, FP ve FN değerlerini hesapla
        TP += ((predicted == labels) & (labels == 1)).sum().item()
        FP += ((predicted != labels) & (labels == 0)).sum().item()
        FN += ((predicted != labels) & (labels == 1)).sum().item()
        TN += ((predicted == labels) & (labels == 0)).sum().item()


accuracy = correct / total

# Kesinlik (Precision) hesapla
precision = TP / (TP + FP)

# Duyarlılık (Recall) hesapla
recall = TP / (TP + FN)

# F1-Skoru hesapla
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

print(f"TP: {TP:.4f}")
print(f"FP: {FP:.4f}")
print(f"FN: {FN:.4f}")
print(f"TN: {TN:.4f}")



Test Accuracy: 0.5597
Precision: 0.6515
Recall: 0.6295
F1-Score: 0.6403
TP: 243.0000
FP: 130.0000
FN: 143.0000
TN: 104.0000


In [13]:
torch.cuda.empty_cache()